<a href="https://colab.research.google.com/github/agentescrossmodais/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
import os
from google.colab import drive  # API nativa do Google Colab
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.model_selection as ms
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.decomposition as decomposition
import pingouin as pg
import io

# --- Configurações de Estilo ---
FIGSIZE = (12, 8)
DEFAULT_PALETTE = "viridis"

sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 20,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# --- Funções Auxiliares ---
def mount_google_drive():
    """Monta o Google Drive usando a API nativa."""
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    """Retorna o caminho completo no Drive."""
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory_exists_on_drive(relative_path):
    """Cria diretório no Drive, se não existir."""
    drive_path = get_drive_path(relative_path)
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"Diretório criado: {drive_path}")
    return drive_path

def create_figure():
    """Cria uma figura com o tamanho padrão."""
    return plt.figure(figsize=FIGSIZE)

def save_fig(fig, filename, drive_folder_path):
    """Salva a figura no Drive."""
    try:
        filepath = os.path.join(drive_folder_path, filename)
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        print(f"Gráfico salvo: {filepath}")
    except Exception as e:
        print(f"Erro ao salvar '{filename}': {e}")
    finally:
        plt.close(fig)

# --- Funções de Visualização Aprimoradas ---
def plot_boxplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota boxplot com estilo personalizado."""
    try:
        fig = create_figure()
        sns.boxplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
        plt.title(title, color='white')
        plt.xlabel(x_col.replace('_', ' '), color='cyan')
        plt.ylabel(y_col.replace('_', ' '), color='cyan')
        if hue:
            plt.legend(title=hue.replace('_', ' '), loc='best', facecolor='black', edgecolor='white')
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_boxplot: {e}")

def plot_violin(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota violin plot com estilo personalizado."""
    try:
        fig = create_figure()
        sns.violinplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE, split=(hue is not None))
        plt.title(title, color='white')
        plt.xlabel(x_col.replace('_', ' '), color='cyan')
        plt.ylabel(y_col.replace('_', ' '), color='cyan')
        if hue:
            plt.legend(title=hue.replace('_', ' '), loc='best', facecolor='black', edgecolor='white')
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_violin: {e}")

def plot_kde(df, col, title, filename, drive_folder_path, hue=None):
    """Plota KDE plot com estilo personalizado."""
    try:
        fig = create_figure()
        if hue:
            for level in df[hue].unique():
                sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True, palette=DEFAULT_PALETTE)
            plt.legend(title=hue.replace('_', ' '), loc='best', facecolor='black', edgecolor='white')
        else:
            sns.kdeplot(df[col], shade=True, color='cyan')
        plt.title(title, color='white')
        plt.xlabel(col.replace('_', ' '), color='cyan')
        plt.ylabel('Densidade', color='cyan')
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_kde: {e}")

# --- Funções de Análise ---
def gerar_dados_teste(n_alunos=40):
    """Gera dados sintéticos para teste."""
    np.random.seed(42)
    dados = pd.DataFrame({
        'aluno_id': range(1, n_alunos + 1),
        'desempenho_prova': np.random.normal(loc=70, scale=15, size=n_alunos),
        'frequencia': np.random.uniform(low=0.7, high=1, size=n_alunos),
        'participacao_atividades': np.random.poisson(lam=3, size=n_alunos),
        'questionario_socioemocional': np.random.randint(1, 6, size=n_alunos),
        'feedback_professor': np.random.normal(loc=4, scale=0.8, size=n_alunos),
        'grupo': np.random.choice(['ComAgentes', 'SemAgentes'], size=n_alunos)  # Alteração aqui
    })
    return dados

def realizar_testes_t(df, variavel_teste, grupo_coluna, drive_folder_path):
    """Realiza testes t independentes entre grupos e gera visualizações."""
    print(f"\n--- Testes t para {variavel_teste} ---")
    if grupo_coluna not in df.columns or variavel_teste not in df.columns:
        print(f"Colunas '{grupo_coluna}' ou '{variavel_teste}' não encontradas.")
        return
    grupos = df[grupo_coluna].unique()
    if len(grupos) < 2:
        print("Número insuficiente de grupos para o teste t.")
        return
    try:
        if not pd.api.types.is_numeric_dtype(df[variavel_teste]):
            df[variavel_teste] = pd.to_numeric(df[variavel_teste], errors='coerce')
            df.dropna(subset=[variavel_teste], inplace=True)
        grupo1 = df[df[grupo_coluna] == 'ComAgentes'][variavel_teste]
        grupo2 = df[df[grupo_coluna] == 'SemAgentes'][variavel_teste]
        ttest_result = pg.ttest(grupo1, grupo2, paired=False)
        print(f"\nTeste t para {variavel_teste} (ComAgentes vs SemAgentes):\n", ttest_result)
        plot_boxplot(df, grupo_coluna, variavel_teste, f"{variavel_teste} - Boxplot",
                     f"{variavel_teste}_boxplot.png", drive_folder_path, hue=grupo_coluna)
        plot_violin(df, grupo_coluna, variavel_teste, f"{variavel_teste} - Violin Plot",
                    f"{variavel_teste}_violin.png", drive_folder_path, hue=grupo_coluna)
        plot_kde(df, variavel_teste, f"{variavel_teste} - KDE", f"{variavel_teste}_kde.png",
                 drive_folder_path, hue=grupo_coluna)
    except Exception as e:
        print(f"Erro no teste t para '{variavel_teste}': {e}")

# --- Função Principal (main) ---
if __name__ == "__main__":
    mount_google_drive()
    drive_base_path = '/content/drive/MyDrive/'
    graficos_drive_path = ensure_directory_exists_on_drive('graficos')
    sumario_drive_path = ensure_directory_exists_on_drive('sumario')
    display(Markdown("# Análise do Sistema de Indicadores Sintéticos (Agentes Crossmodais)"))

    # --- Carregar Dados ---
    df = gerar_dados_teste()

    # --- Pré-processar Dados ---
    df = pre_processar_dados(df)

    # --- Análise Exploratória e Relatório ---
    gerar_relatorio_estatistico(df, sumario_drive_path)

    # --- Definir colunas ---
    colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
    colunas_fatoriais = ['desempenho_prova', 'frequencia', 'participacao_atividades', 'questionario_socioemocional', 'feedback_professor']
    colunas_cluster = ['Desempenho_Academico', 'Engajamento_Escolar', 'Bem_Estar_Socioemocional']
    grupo_coluna = 'grupo'

    # --- Análises ---
    analisar_correlacoes(df, colunas_numericas, graficos_drive_path)
    df = aplicar_analise_fatorial(df, colunas_fatoriais, graficos_drive_path)

    if all(col in df.columns for col in colunas_cluster):
        df = aplicar_analise_cluster(df, colunas_cluster, graficos_drive_path)

    if grupo_coluna in df.columns:
        for col in colunas_numericas:
            if col not in ['Desempenho_Academico', 'Engajamento_Escolar', 'Bem_Estar_Socioemocional', 'cluster', 'aluno_id']:
                realizar_testes_t(df, col, grupo_coluna, graficos_drive_path)

    display(Markdown("## Conclusões"))
    display(Markdown("Resultados da análise do sistema de indicadores sintéticos."))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Análise do Sistema de Indicadores Sintéticos (Agentes Crossmodais)

Pré-processamento dos dados...
Gerando relatório estatístico...
Relatório estatístico salvo em: /content/drive/MyDrive/sumario/sumario.txt
Analisando correlações...

Matriz de Correlação:
                              aluno_id  desempenho_prova  frequencia  \
aluno_id                     1.000000         -0.250444   -0.112745   
desempenho_prova            -0.250444          1.000000    0.057739   
frequencia                  -0.112745          0.057739    1.000000   
participacao_atividades      0.014239          0.278820    0.011033   
questionario_socioemocional  0.171382         -0.002671    0.060004   
feedback_professor          -0.160473          0.411418   -0.021641   

                             participacao_atividades  \
aluno_id                                    0.014239   
desempenho_prova                            0.278820   
frequencia                                  0.011033   
participacao_atividades                     1.000000   
questionario_socioemocional      


        **Interpretação:**
        - **Fator 1 (Desempenho Acadêmico):** Representa o desempenho geral em provas e atividades acadêmicas.
        - **Fator 2 (Engajamento Escolar):** Reflete o envolvimento do aluno nas atividades escolares.
        - **Fator 3 (Bem-Estar Socioemocional):** Indica o bem-estar emocional e social do aluno.
        


--- Análise de Cluster (K-Means) ---

Distribuição dos Clusters:
 cluster
0    14
2    12
3     8
1     6
Name: count, dtype: int64

Características dos Clusters (Médias):
          Desempenho_Academico  Engajamento_Escolar  Bem_Estar_Socioemocional
cluster                                                                     
0                   -0.562797            -0.336891                 -0.087074
1                    1.145879             0.805695                 -0.120058
2                    0.601587            -0.485806                  0.067105
3                   -0.776896             0.713998                  0.141765
Gráfico salvo: /content/drive/MyDrive/graficos/clusters_kmeans.png

--- Testes t para desempenho_prova ---

Teste t para desempenho_prova (ComAgentes vs SemAgentes):
                T        dof alternative     p-val          CI95%   cohen-d  \
T-test -0.300078  37.904673   two-sided  0.765757  [-10.52, 7.8]  0.093871   

         BF10     power  
T-test  0.321 

...

Gráfico salvo: /content/drive/MyDrive/graficos/feedback_professor_violin.png


<ipython-input-14-9a9d93c62015>:110: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True, palette=DEFAULT_PALETTE)
<ipython-input-14-9a9d93c62015>:110: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True, palette=DEFAULT_PALETTE)
<ipython-input-14-9a9d93c62015>:110: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True, palette=DEFAULT_PALETTE)
<ipython-input-14-9a9d93c62015>:110: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True, palette=DEFAULT_PALETTE)


Gráfico salvo: /content/drive/MyDrive/graficos/feedback_professor_kde.png


## Conclusões

Resultados da análise do sistema de indicadores sintéticos.